# Review: R-CNN (Regions with CNN features, Object Detection)

* SH Tsang의 [Review: R-CNN (Object Detection)](https://medium.com/coinmonks/review-r-cnn-object-detection-b476aba290d1)를 번역한 글입니다.

* [2014 CVPR] [R-CNN] [Rich feature hierarchies for accurate object detection and semantic segmentation](https://arxiv.org/pdf/1311.2524.pdf)

---

## Introduction

Region-CNN(R-CNN)[1]은 CNN 기반의 Obeject detection 방법론 중 하나이다. 이 방법을 기본으로 object detection 의 속도를 향상시킨 <strong>fast R-CNN, faster R-CNN'</strong>  알고리즘, object instacnce segmentation 분야의 <strong>mask R-CNN</strong>이 개발되었다. 하지만 <strong>YOLO, SSD</strong>과 같은 object detection approach와는 다르다고 할 수 있다.

딥러닝을 이용한 사물 인식 방법 중에 R-CNN은 꼭 읽어봐야 할 논문으로 알려져 있다. 이 논문은 2014년 CVPR 논문이며, 약 6000회의 인용수를 보여준다.

<b>사물 인식을 하기위해서는 사물(object)의 class와 bounding box size, location을 알고 있어야 한다.</b> 즉 training data에 해당 내용이 포함되어야 한다.

일반적으로 기존의 object detection 방법은 다음과 같다('OverFeat' uses a sliding-window). 각 이미지에 아래 사진처럼 위치를 찾기위한 <b>sliding window</b>가 존재하고, 이 window를 움직이면서 사물을 찾는 단순한 방법을 사용한다. 하지만, 같은 물체라도 카메라의 거리나 물체의 크기로 인해 다양한 비율(aspect ratios), 크기(sizes)를 가지고 있으며, 이미지의 크기가 다른 경우도 있어 window size를 잘 결정해야만 좋은 모델을 만들 수 있다. 이 방벙을 쓰는 동시에, 각 위치(each location)에서 CNN으로 classification을 사용한다면 속도도 매우 느릴 것이다. <br>

<center>
<img src="https://cdn-images-1.medium.com/max/1600/1*feg0v9MYMkIDqfa1zWBBzA.png" width="600"><center><br><i>
Illustration of Sliding Window (Left) with Different Aspect Ratios and Sizes (Right)
</i></center></img><br>

## R-CNN Process

R-CNN의 과정(process)은 다음과 같다.

1. 먼저, R-CNN은 'selective search[2]'를 사용해서 약 2000개의 region proposals(i.e bounding boxes for image classification, category-independent region proposals)를 생성한다.

2. 그 다음, 각 bounding box에서 CNN으로 classification을 수행한다.

3. 마지막으로, 각 bounding box를 regression을 이용해서 정제(refined)한다.

논문에서는 R-CNN이 3가지 모듈로 구성되었다고 한다.
<i>
1. The first generates category-independent region proposals. These proposals define the set of candidate detections available to our detector. 
2. he second module is a large convolutional neural network that extracts a fixed-length feature vector from each region.
3. The third module is a set of classspecific linear SVMs.</i>


<img src="https://cdn-images-1.medium.com/max/1600/1*CI8tVwe1QIj1Wknh6ZuLWA.png" width= "600"><br><center><i>
    R-CNN Flowchart
</i></center></img>

## What will be covered:

1. Selective Search
2. CNN-based Classification and Scoring
3. Results
4. Conclusions

## 1. Selective Search

<center>
<img src="https://cdn-images-1.medium.com/max/1600/1*NXZoM83IKAM9NZzRTJk1jw.png" width= "600"><br><center><i>
    Selective Search
</i></center></img>
</center>

Selective search의 자세한 내용은 [2] Reference를 참고하면 된다.

1. 먼저, color similarities, texture similarities, region size, region filling가 non-object-based segmentation를 수행하기 위해 사용된다. 위 그림에서 보듯이 많은 수의 작은 영역(segmented area)를 얻는다.

2. 다음으로, bottom-up approach를 사용해서 작은 영역을 합쳐 큰 영역(large segmented)으로 만든다.

3. 이렇게해서 약 2000개의 region proposals(bounding box candidiates)이 만들어 지게 된다.

## 2. CNN-based Classification and Scoring

<img src="https://cdn-images-1.medium.com/max/1600/1*Sequfmhm-iytuxqBjq3kDg.png" width= "600"><br><center><i>
    R-CNN Flowchart with More Details
</i></center></img><br>

CNN feature를 얻기 위해서 pre-trained된 AlexNet을 사용한다. 

각 박스영역(proposal)의 227x227 RGB 이미지(mean-subtracted)를 5개의 conv layer와 2개의 fully connetecd layer를 통과시키면 4096 차원의 feature vector가 계산된다. Input은 bounding box의 크기(size)와 모양(shape)이 여러개지만, 227x227로 고정 시킨다(그래서 작은 bouding box의 픽셀들은 왜곡이 발생하게 된다).

Feature vector는 미리 class에 대해 학습된 SVM을 이용해서 판단(scored)한다(논문에서는 SVM을 이용하는 방법과 softmax를 통한 기존 CNN output의 결과를 비교했고, SVM을 선택했다고 한다).

각 class에서 bounding box에 대해 High Iou(intersection over Union, 중복된 부분을 판단하는 기준)를 보이면 같은 object를 포함하고 있다고 판단해서 버리게(rejected)된다.

예측한 bounding box는 box regressor(원본 bounding box와 얼마나 가까운지 판단하는 regressor)를 이용해서 fine-tuned 과정을 추가할 수도 있다.

## 3. Results

### 3.1 VOC 2010

2010 VOC에서 R-CNN와 R-CNN BB가 가장 높은 mAP를 기록했다. BB는 'bounding-box regression'을 사용한 R-CNN 이다.

<img src="https://cdn-images-1.medium.com/max/1600/1*CbpKWiVsB-beWNgVGoQ6zg.png" width= "600"><br><center><i>
    VOC 2010
</i></center></img>

### 3.2 ILSVRC 2013

R-CNN BB는 'OverFeat'을 능가하는 성능을 보여주었으며, 2013 ILSVRC localization task에서 1위를 달성했다.<br>

<img src="https://cdn-images-1.medium.com/max/1600/1*CFjNHMUtq4uBAEbKOzaRbg.png" width= "600"><br><center><i>
    Some Amazing ILSVRC 2013 Results
</i></center></img><br>

<img src="https://cdn-images-1.medium.com/max/1600/1*0dNYXOVpiXwjFv0GsWVhGw.png" width= "600"><br><center><i>
    Some ILSVRC 2013 Results with Some Missing Detections
</i></center></img><br>

<img src="https://cdn-images-1.medium.com/max/1600/1*gNrrvXcMlcqp8Ueg3j92-g.png" width= "500"><br><center><i>
    ILSVRC 2013
</i></center></img>

### 3.3 VOC 2007
<br>
<img src="https://cdn-images-1.medium.com/max/1600/1*FsBzLo1WYxBTs43S2LULFw.png" width= "600"><br><center><i>
    Some examples with high activations in VOC 2007
</i></center></img><br>

<img src="https://cdn-images-1.medium.com/max/1600/1*6hXU8VS9uyeWr6zFYuUdrQ.png" width= "600"><br><center><i>
    VOC 2007
</i></center></img>

## 4. Conclusions

R-CNN은 image classification에서 사용하던 CNN을 처음으로 object detection에 적용한 모델이다. 하지만 이 모델의 성능은 Bounding box가 어떻게 생성되는가에 달려있다. Bounding box가 object를 하나도 담지 못한다면 의미없는 모델이 되기 때문이다. 

논문에는 CNN으로 AlexNet을 사용했지만, 이후에 나온 다른 CNN 구조(VGGNet, GoolgNet)을 사용하면 더 좋은 결과를 얻을 수 있다.

## References
1. [2014 CVPR] [R-CNN]<br>
Rich feature hierarchies for accurate object detection and semantic segmentation
2. [2013 IJCV] [Selective Search]<br>
Selective Search for Object Recognition
3.[2012 NIPS] [AlexNet]<br>
ImageNet Classification with Deep Convolutional Neural Networks
4. [2014 ICLR] [OverFeat]<br>
OverFeat: Integrated Recognition, Localization and Detection using Convolutional Networks
5. [2015 ICLR] [VGGNet]<br>
Very Deep Convolutional Networks for Large-Scale Image Recognition